In [0]:
%pip install databricks-agents mlflow>=3.1.0 databricks-sdk==0.55.0 unitycatalog-ai[databricks]
# Restart to load the packages into the Python environment
dbutils.library.restartPython()

INFO: pip is looking at multiple versions of databricks-agents to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of databricks-agents to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.0 MB/s eta 0:00:00
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of mlflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mlflow[databricks] to determine which version is compatible with other requirements. This could take a w

In [0]:
# %run ../_resources/01-setup

USE CATALOG `main_build`
using catalog.database `main_build`.`dbdemos_ai_agent`


folder doesn't exists, generating the data...
saving /Volumes/main/dbdemos_ai_agent/raw_data/customers/part-00000-tid-8191709100379501368-970ccff7-dbf7-4835-b528-11f44a5cee3f-132774-1.c000.snappy.parquet
saving /Volumes/main/dbdemos_ai_agent/raw_data/subscriptions/part-00000-tid-3580466118653320274-6001ce3c-cb6c-4343-84c5-68d58f9b9b7e-132783-1.c000.snappy.parquet
saving /Volumes/main/dbdemos_ai_agent/raw_data/billing/part-00000-tid-4865906735622801564-2246f189-7614-427e-9ad9-222bf6db55b7-132792-1.c000.snappy.parquet
saving /Volumes/main/dbdemos_ai_agent/raw_data/eval_dataset/part-00006-tid-7161042216793854883-97211a65-6f05-4793-800e-f83cd92c5235-56437-1-c000.json
saving /Volumes/main/dbdemos_ai_agent/raw_data/eval_dataset/part-00001-tid-7161042216793854883-97211a65-6f05-4793-800e-f83cd92c5235-56441-1-c000.json
saving /Volumes/main/dbdemos_ai_agent/raw_data/eval_dataset/part-00002-tid-7161042216793854883-97211a65-6f05-4793-800e-f83cd92c5235-56436-1-c000.json
saving /Volumes/main/dbdemos

## Configuration file

Please change your catalog and schema here to run the demo on a different catalog.

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=984752964297111&notebook=%2Fconfig&demo_name=ai-agent&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fai-agent%2Fconfig&version=1&user_hash=877940098f7eb5f56ae4a086f4dad965c79afbcfa1d67e1964c0aace55684a18">


# Technical Setup notebook. Hide this cell results
Initialize dataset to the current user and cleanup data when reset_all_data is set to true

Do not edit

USE CATALOG `main`
using catalog.database `main`.`dbdemos_ai_agent`


To speedup the demo, we won't generate the data but download it directly. IF it fails you can run the generation notebooks in the _resource folders instead.

folder doesn't exists, generating the data...
saving /Volumes/main/dbdemos_ai_agent/raw_data/customers/part-00000-tid-8191709100379501368-970ccff7-dbf7-4835-b528-11f44a5cee3f-132774-1.c000.snappy.parquet
saving /Volumes/main/dbdemos_ai_agent/raw_data/subscriptions/part-00000-tid-3580466118653320274-6001ce3c-cb6c-4343-84c5-68d58f9b9b7e-132783-1.c000.snappy.parquet
saving /Volumes/main/dbdemos_ai_agent/raw_data/billing/part-00000-tid-4865906735622801564-2246f189-7614-427e-9ad9-222bf6db55b7-132792-1.c000.snappy.parquet
saving /Volumes/main/dbdemos_ai_agent/raw_data/eval_dataset/part-00004-tid-7161042216793854883-97211a65-6f05-4793-800e-f83cd92c5235-56443-1-c000.json
saving /Volumes/main/dbdemos_ai_agent/raw_data/eval_dataset/part-00003-tid-7161042216793854883-97211a65-6f05-4793-800e-f83cd92c5235-56442-1-c000.json
saving /Volumes/main/dbdemos_ai_agent/raw_data/eval_dataset/part-00007-tid-7161042216793854883-97211a65-6f05-4793-800e-f83cd92c5235-56445-1-c000.json
saving /Volumes/main/dbdemos

Generate synthetic data we're going to use with our agents.


## 1. Get the customer details based on their email

Let's add a function to retrieve a customer detail based on their email.

*Note: realtime postgres tables could be used to provide low latencies for point-queries like these. However, to keep this demo simple, we'll use a SQL endpoint. Using a SQL endpoint is also a very good way to provide analytics capabilities to your agent!*


In [0]:
%sql 
USE CATALOG <insert catalog here>;
USE SCHEMA <insert schema here;

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_exposure_by_country(country_input STRING COMMENT 'input used to retrieve investment information based on country, it only takes in HK, US, MU')
RETURNS TABLE (
    `Source Price Date` STRING,
    `Company Code` STRING,
    `Company Name` STRING,
    `Portfolio Name` STRING,
    `Country of Risk` STRING,
    `Industry Sector Name` STRING,
    `Long Name` STRING,
    `GICS code (6 digit)` STRING,
    `Debt/Equity` STRING,
    `Asset Type Name` STRING,
    `Market Value in RC` DOUBLE
)
COMMENT 'Returns data about exposure of AIA investment matching the provided country. Long Name column may have the segment/industry/sector'
RETURN (
    SELECT * FROM `catalog`.`schema`.`table`
    WHERE `Country of Risk` = country_input
);


In [0]:
%sql
CREATE OR REPLACE FUNCTION get_unrealized_gains_by_country(country_input STRING COMMENT 'input used to retrieve investment information based on country, it only takes in HK, US, MU')
RETURNS TABLE (
    `Source Price Date` STRING,
    `Company Code` STRING,
    `Company Name` STRING,
    `Portfolio Name` STRING,
    `Country of Risk` STRING,
    `Industry Sector Name` STRING,
    `Long Name` STRING,
    `GICS code (6 digit)` STRING,
    `Debt/Equity` STRING,
    `Asset Type Name` STRING,
    `Unrealized Gain Loss OCI - Market in RC` DOUBLE,
    `Unrealized Gain Loss P&L - Market in RC` DOUBLE,
    `Unrealized Gain Loss P&L - FX in RC` DOUBLE,
    `Total Unrealized Gain Loss OCI in RC` DOUBLE,
    `Total Unrealized Gain Loss P&L in RC` DOUBLE
)
COMMENT 'Returns data about unrealized gains of AIA investment matching the provided country. Long Name column may have the segment/industry/sector'
RETURN (
    SELECT * FROM `catalog`.`schema`.`table`
    WHERE `Country of Risk` = country_input
);


In [0]:
%sql SELECT * FROM get_exposure_by_country('Hong Kong');

Source Price Date,Company Code,Company Name,Portfolio Name,Country of Risk,Industry Sector Name,Long Name,GICS code (6 digit),Debt/Equity,Asset Type Name,Market Value in RC


In [0]:
print(catalog, dbName)

main dbdemos_ai_agent
